<a href="https://colab.research.google.com/github/jocelynbaduria/Assignment-5_part_abcd/blob/main/Jocelyn_Baduria_cmpe258_Assignment5d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### D. XGBoost Model(rmse=3.99031)


Load data from Github

In [55]:
!wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv

--2021-04-25 21:15:07--  https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922812 (1.8M) [text/plain]
Saving to: ‘data.csv.1’

data.csv.1          100%[===================>]   1.83M  --.-KB/s    in 0.03s   

2021-04-25 21:15:07 (66.2 MB/s) - ‘data.csv.1’ saved [1922812/1922812]



Import Libraries

In [56]:
# load some default Python modules
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

plt.style.use('seaborn-whitegrid')

import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb

In [57]:
taxi_data = pd.read_csv('/content/data.csv')
taxi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   pickup_community_area   15000 non-null  float64
 1   fare                    15002 non-null  float64
 2   trip_start_month        15002 non-null  int64  
 3   trip_start_hour         15002 non-null  int64  
 4   trip_start_day          15002 non-null  int64  
 5   trip_start_timestamp    15002 non-null  int64  
 6   pickup_latitude         15000 non-null  float64
 7   pickup_longitude        15000 non-null  float64
 8   dropoff_latitude        14519 non-null  float64
 9   dropoff_longitude       14519 non-null  float64
 10  trip_miles              15002 non-null  float64
 11  pickup_census_tract     1 non-null      float64
 12  dropoff_census_tract    10761 non-null  float64
 13  payment_type            15002 non-null  object 
 14  company                 9862 non-null 

Data - Preprocessing

In [58]:
taxi_data.head()

,pickup_community_area,fare,trip_start_month,trip_start_hour,trip_start_day,trip_start_timestamp,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_miles,pickup_census_tract,dropoff_census_tract,payment_type,company,trip_seconds,dropoff_community_area,tips
0,NaN,12.45,5,19,6,1400269500,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,0.0,NaN,0.0
1,NaN,0.00,3,19,5,1362683700,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Unknown,Chicago Elite Cab Corp.,300.0,NaN,0.0
2,60.0,27.05,10,2,3,1380593700,41.836150,-87.648788,NaN,NaN,12.6,NaN,NaN,Cash,Taxi Affiliation Services,1380.0,NaN,0.0
3,10.0,5.85,10,1,2,1382319000,41.985015,-87.804532,NaN,NaN,0.0,NaN,NaN,Cash,Taxi Affiliation Services,180.0,NaN,0.0
4,14.0,16.65,5,7,5,1369897200,41.968069,-87.721559,NaN,NaN,0.0,NaN,NaN,Cash,Dispatch Taxi Affiliation,1080.0,NaN,0.0


In [59]:
# Get the count, mean and stdev, min, max, quantile values
taxi_data.describe()

,pickup_community_area,fare,trip_start_month,trip_start_hour,trip_start_day,trip_start_timestamp,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_miles,pickup_census_tract,dropoff_census_tract,trip_seconds,dropoff_community_area,tips
count,15000.000000,15002.000000,15002.000000,15002.000000,15002.000000,1.500200e+04,15000.000000,15000.000000,14519.000000,14519.000000,15002.000000,1.000000e+00,1.076100e+04,14996.000000,14495.000000,15002.000000
mean,22.250267,11.768216,6.585655,13.632316,4.186642,1.408495e+09,41.903046,-87.657551,41.902672,-87.654113,2.872820,1.703108e+10,1.703135e+10,777.627501,20.967782,1.076674
std,19.414828,11.538850,3.390997,6.620927,2.015694,2.916043e+07,0.037751,0.067846,0.038478,0.056616,15.276007,NaN,3.312243e+05,977.538769,17.641056,2.158340
min,1.000000,0.000000,1.000000,0.000000,1.000000,1.357000e+09,41.694879,-87.913625,41.663671,-87.913625,0.000000,1.703108e+10,1.703101e+10,0.000000,1.000000,0.000000
25%,8.000000,5.850000,4.000000,9.000000,2.000000,1.384622e+09,41.880994,-87.655998,41.880994,-87.656804,0.000000,1.703108e+10,1.703108e+10,360.000000,8.000000,0.000000
50%,8.000000,7.850000,7.000000,15.000000,4.000000,1.407260e+09,41.892508,-87.633308,41.893216,-87.634156,1.000000,1.703108e+10,1.703124e+10,540.000000,12.000000,0.000000
75%,32.000000,12.450000,10.000000,19.000000,6.000000,1.431339e+09,41.921877,-87.626211,41.922686,-87.626215,2.500000,1.703108e+10,1.703183e+10,960.000000,32.000000,2.000000
max,77.000000,700.070000,12.000000,23.000000,7.000000,1.483116e+09,42.009623,-87.572782,42.021224,-87.540936,1710.000000,1.703108e+10,1.703198e+10,72120.000000,77.000000,47.000000


In [60]:
##Get unique count for each column
taxi_data.nunique()

pickup_community_area        62
fare                        423
trip_start_month             12
trip_start_hour              24
trip_start_day                7
trip_start_timestamp      13789
pickup_latitude             236
pickup_longitude            236
dropoff_latitude            291
dropoff_longitude           291
trip_miles                  530
pickup_census_tract           1
dropoff_census_tract        229
payment_type                  7
company                      61
trip_seconds                110
dropoff_community_area       69
tips                        462
dtype: int64

In [61]:
taxi_data = taxi_data.drop(['trip_start_timestamp','trip_miles','pickup_census_tract','dropoff_census_tract','trip_seconds','payment_type','tips', 
              'company','dropoff_community_area','pickup_community_area'], axis=1)
taxi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fare               15002 non-null  float64
 1   trip_start_month   15002 non-null  int64  
 2   trip_start_hour    15002 non-null  int64  
 3   trip_start_day     15002 non-null  int64  
 4   pickup_latitude    15000 non-null  float64
 5   pickup_longitude   15000 non-null  float64
 6   dropoff_latitude   14519 non-null  float64
 7   dropoff_longitude  14519 non-null  float64
dtypes: float64(5), int64(3)
memory usage: 937.8 KB


In [62]:
#Drop rows with N/A 
taxi_data = taxi_data.dropna()

#Drop the rows where FARE > 200( only 1 row found )
taxi_data = taxi_data.drop(taxi_data[taxi_data['fare'] > 200].index)

#Change the datatype of month, hour day feature to string 
taxi_data['trip_start_month'] = taxi_data['trip_start_month'].astype('str')
taxi_data['trip_start_hour'] = taxi_data['trip_start_hour'].astype('str')
taxi_data['trip_start_day'] = taxi_data['trip_start_day'].astype('str')
taxi_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14518 entries, 28 to 15000
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fare               14518 non-null  float64
 1   trip_start_month   14518 non-null  object 
 2   trip_start_hour    14518 non-null  object 
 3   trip_start_day     14518 non-null  object 
 4   pickup_latitude    14518 non-null  float64
 5   pickup_longitude   14518 non-null  float64
 6   dropoff_latitude   14518 non-null  float64
 7   dropoff_longitude  14518 non-null  float64
dtypes: float64(5), object(3)
memory usage: 1020.8+ KB


Prepare Train test split sample from Original Taxi dataset

In [63]:
np.random.seed(seed=2)
sample = np.random.rand(len(taxi_data)) < 0.95
traindf = taxi_data[sample]
testdf = taxi_data[~sample]

print(len(traindf))
print(len(testdf))

traindf.to_csv("train.csv", index=False, header=True)
testdf.to_csv("test.csv", index=False, header=True)

13804
714


In [64]:
X = pd.read_csv('/content/train.csv')
Y = pd.read_csv('/content/train.csv')

In [65]:
params = {
   
    'max_depth': 7,
    'gamma' :0,
    'eta':.03, 
    'subsample': 1,
    'colsample_bytree': 0.9, 
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 0
}

In [66]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X.drop('fare', axis=1), Y['fare'], test_size=0.2, random_state = 42)

In [67]:
def XGBmodel(X_train,X_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_test = xgb.DMatrix(X_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=5000, 
                    early_stopping_rounds=10,evals=[(matrix_test,'test')])
    return model

model = XGBmodel(X_train,X_test,y_train,y_test,params)

[21:15:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	test-rmse:13.4735
Will train until test-rmse hasn't improved in 10 rounds.
[1]	test-rmse:13.1124
[2]	test-rmse:12.7578
[3]	test-rmse:12.4182
[4]	test-rmse:12.0925
[5]	test-rmse:11.7748
[6]	test-rmse:11.4738
[7]	test-rmse:11.1748
[8]	test-rmse:10.8869
[9]	test-rmse:10.6079
[10]	test-rmse:10.3407
[11]	test-rmse:10.0874
[12]	test-rmse:9.83733
[13]	test-rmse:9.59854
[14]	test-rmse:9.3696
[15]	test-rmse:9.14453
[16]	test-rmse:8.93044
[17]	test-rmse:8.72467
[18]	test-rmse:8.51742
[19]	test-rmse:8.32668
[20]	test-rmse:8.13424
[21]	test-rmse:7.95809
[22]	test-rmse:7.78005
[23]	test-rmse:7.61661
[24]	test-rmse:7.46028
[25]	test-rmse:7.30537
[26]	test-rmse:7.16091
[27]	test-rmse:7.02387
[28]	test-rmse:6.88943
[29]	test-rmse:6.76075
[30]	test-rmse:6.63264
[31]	test-rmse:6.50932
[32]	test-rmse:6.39107
[33]	test-rmse:6.27947
[34]	test-rmse:6.17128
[35]	test-rmse:6.06382

In [68]:
prediction = model.predict(xgb.DMatrix(X_test), ntree_limit = model.best_ntree_limit).tolist()

In [69]:
prediction

[6.104604721069336,
 37.4227294921875,
 7.933529853820801,
 6.246888637542725,
 6.292518615722656,
 8.34505558013916,
 35.932708740234375,
 6.283674240112305,
 15.427735328674316,
 9.020915985107422,
 6.866586685180664,
 9.2535400390625,
 6.103100299835205,
 6.626970291137695,
 36.07346725463867,
 10.486266136169434,
 7.925539970397949,
 7.413977146148682,
 17.08866310119629,
 7.046564102172852,
 6.308060169219971,
 6.50618314743042,
 6.345444679260254,
 9.291860580444336,
 9.747477531433105,
 6.162304878234863,
 17.83365249633789,
 12.473044395446777,
 12.874483108520508,
 8.687751770019531,
 6.271297454833984,
 7.443894863128662,
 6.4634690284729,
 8.255441665649414,
 6.668532371520996,
 6.947591781616211,
 6.242298603057861,
 9.343461036682129,
 8.501598358154297,
 8.26611328125,
 7.009871482849121,
 6.15297794342041,
 6.353793621063232,
 8.519640922546387,
 8.974813461303711,
 6.755751132965088,
 15.355325698852539,
 6.445568084716797,
 6.8324875831604,
 7.774332523345947,
 7.97009